In [10]:
import os
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image as OpenpyxlImage

# 配置tesseract的路径
pytesseract.pytesseract.tesseract_cmd = r'D:\software\Tesseract-OCR\tesseract.exe'

# 指定PDF文件夹路径和Excel模板路径
pdf_folder_path = './pdf'
excel_template_path = 'template.xlsx'
output_excel_path = 'output.xlsx'
temp_img_path = "./temp_img" 

# 加载Excel模板
wb = load_workbook(excel_template_path)
ws = wb.active

"""
    解析文本，提取个人信息和测试数据。这里需要根据OCR提取的文本格式进行调整。
"""
def parse_text(text):
    
    # 示例解析逻辑，需要根据实际文本格式调整
    personal_info = {}
    test_data = {}
    # 假设个人信息和测试数据通过特定标识符分隔
    sections = text.split('测试数据开始')
    personal_info_section = sections[0]
    test_data_section = sections[1] if len(sections) > 1 else ''
    
    # 解析个人信息
    # 示例：personal_info['测试号'] = ...
    
    # 解析测试数据
    # 示例：test_data['VT预计值'] = ...
    
    return personal_info, test_data

# 遍历PDF文件夹
for pdf_file in sorted(os.listdir(pdf_folder_path)):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, pdf_file)
        doc = fitz.open(pdf_path)
        page = doc[0]  # 假设每个PDF只有一页
        
        # 提取整页作为图像
        pix = page.get_pixmap()
        image_path = os.path.join(temp_img_path, 'temp_image.jpg')
        pix.save(image_path)
        
        # 使用OCR提取文本
        text = pytesseract.image_to_string(Image.open(image_path), lang='chi_sim+eng')
        print(text)
        """ # 解析文本
        personal_info, test_data = parse_text(text)
        
        # 将提取的信息填充到Excel中
        # 这里需要根据Excel模板的具体格式进行调整
        max_row = ws.max_row + 1
        ws[f'A{max_row}'] = personal_info.get('测试号', '')
        # 继续填充其他个人信息和测试数据
        
        # 将测试曲线图插入到Excel
        img = OpenpyxlImage(image_path)
        img.anchor = f'Z{max_row}'  # 假设测试曲线图插入到Z列
        ws.add_image(img)

# 保存填充后的Excel文件
wb.save(output_excel_path) """

In [20]:
import pytesseract
from PIL import Image
import os
from pdf2image import convert_from_path
import cv2
pdf_folder_path = './pdf/'
excel_template_path = 'template.xlsx'
output_excel_path = 'output.xlsx'
temp_img_path = "./temp_img/" 

""" 
    用于将pdf文件夹中的文件逐个提取成为图片 
    pdf_folder_path: pdf文件夹的路径
    return: images，提取出来的图像列表
"""
def pdf2img(pdf_folder_path):
    # 初始化列表用于存储从pdf文件中提取出来的图片
    images = []
    # for循环遍历pdf文件夹中的每个pdf文件并将其转换为原始图像存入列表
    for pdf_file in sorted(os.listdir(pdf_folder_path)):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, pdf_file)
            image_list_pdf = convert_from_path(pdf_path)
            image = image_list_pdf[0]
            images.append(image)     
    return images

""" 
    裁剪图片函数
    images: 图像列表
    return: img_cropped_list，裁剪后的图像列表
"""
def crop_images(images):
    # 初始化一个列表，用于存储裁剪后的图像
    img_cropped_list = []
    left = 245 # 左边界
    top = 350    # 上边界
    right = 1355  # 右边界
    bottom = 1500 # 下边界
    for i, image in enumerate(images):
        # 将图片保存以便进行OCR识别（也可以直接对PIL.Image对象进行处理）
        image.save(temp_img_path+f"page_{i+1}.png")

        # 打开原图片开始进行裁剪
        img = Image.open(temp_img_path+f"page_{i+1}.png")
        # 确保裁剪区域的坐标不会导致图像超出范围
        left = max(0, left)
        top = max(0, top)
        right = min(img.width, right)
        bottom = min(img.height, bottom)
        # 开始裁剪
        img_cropped = img.crop((left, top, right, bottom))
        img_cropped.save(temp_img_path+f"page_{i+1}_cropped.png")
        img_cropped_list.append(img_cropped)
    return img_cropped_list

# 新增图片二值化函数
def adaptive_binarize_image(img_path, new_filename=None, display=False):
    # 读取图片为灰度图像
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # 自适应阈值二值化
    block_size = 15
    constant = 5
    adaptive_method = cv2.ADAPTIVE_THRESH_MEAN_C
    threshold_type = cv2.THRESH_BINARY
    img_adaptive_thresholded = cv2.adaptiveThreshold(img, 255, adaptive_method, threshold_type, block_size, constant)

    # 如果没有提供新的文件名，则默认添加"_binary.png"
    if new_filename is None:
        new_filename = os.path.splitext(os.path.basename(img_path))[0] + '_binary.png'
    else:
        # 确保提供了扩展名
        if not new_filename.endswith('.png'):
            new_filename += '.png'

    # 获取当前工作目录
    current_dir = os.getcwd()

    # 构建输出路径
    output_path = os.path.join(current_dir, new_filename)

    # 保存二值化后的图片
    cv2.imwrite(output_path, img_adaptive_thresholded)

    # 显示二值化后的图片（仅当display参数为True时）
    if display:
        cv2.imshow("Adaptive Binarized Image", img_adaptive_thresholded)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

""" 
    使用Tesseract进行OCR识别 
    img_cropped_list: 裁剪后的图像列表
    return: text_list，识别的文本列表
""" 
def ocr(img_cropped_list):
    # 初始化一个文本列表用于存储每张图像提取出来的文本
    text_list = []
    # 用于存储两部分信息的列表
    person_info_list = []
    test_data_list = []

    for i, img_cropped in enumerate(img_cropped_list):
        # 二值化处理
        img_cropped.save(temp_img_path+f"page_{i+1}_cropped.png")
        adaptive_binarize_image(temp_img_path+f"page_{i+1}_cropped.png")
        # 使用Tesseract进行OCR识别
        text = pytesseract.image_to_string(img_cropped, lang='eng+chi_sim')
        text_list.append(text)
    
    # 分割用户信息与测试数据
    for i, text in enumerate(text_list):
        lines = text.splitlines()
        person_info = "\n".join(lines[:6])
        test_data = "\n".join(lines[6:])
        person_info_list.append(person_info)
        test_data_list.append(test_data)

    return person_info_list, test_data_list

def main():
    images = []
    images = pdf2img(pdf_folder_path)
    img_cropped_list = crop_images(images)
    person_info_list, test_data_list = ocr(img_cropped_list)
    
    for person_info in person_info_list:
        print(person_info)
    print("--------------------------------------------------------")
    for test_data in test_data_list:
        print(test_data)


if __name__ == '__main__':
    main()

姓名 : as 测试 号
住院 号 : 1973315 临床 印象 ， 16889648-19
出 生 目 期 : 1957/11/30 操作 者 : ”
: 160 cm Tk.
年 龄 : 66 岁 体重 : 73 kg
预计 实测 %( 实 / 预 )
--------------------------------------------------------
测试 日 其 24/3/04
测试 时 间 16:05:20
ua i) 08 Om ike
BF {1/min] 20. 00 53. 44 267. 2
MV [LVmin] 10.43 38.67 370.8
ERV [L] 0.70 0. 49 70.4
VC MAX [L] 2:59 2. 32 89.5
FVC {L] 2.48 2.17 87.4
FEV 1 [Bui] 2,0 1.64 79.3
FEV 1 % FVC [%] 75. 68
FEV 1 % VC MAX [%] 76.56 70.78 92.5
PEF ti/sl] Sel 5. 06 88.6
MEF 75 {L/s] 5.10 3.87 75.9
MEF 50 [L/s] 3.43 1.43 41.7
MEF 25 (Ste) pe 0. 46 40.6
MMEF 75/25 [L/s] 2.68 0.98 36.7
MVV [L/min] 84.80 56.96 67.2
FEV 1#30 [L/min] 84.80 49.26 58. 1
RV-SB (fii] eh 1.49 16.3
TLC-SB (il ae 3.49 1382)
RV%TLC-SB [%] 41.40 42. 64 103.0
FRC-SB [elie 65 1.98 74.8
FRC%TLC-SB [%] 55. 66 56. 70 101.9
DLCO SB [mmol/min/kPa] 7.11 6. 27 88.1



In [3]:
import pytesseract
from PIL import Image
import os
from pdf2image import convert_from_path
pdf_folder_path = './pdf/'
excel_template_path = 'template.xlsx'
output_excel_path = 'output.xlsx'
temp_img_path = "./temp_img/" 

""" 
    用于将pdf文件夹中的文件逐个提取成为图片 
    pdf_folder_path: pdf文件夹的路径
    return: images，提取出来的图像列表
"""
def pdf2img(pdf_folder_path):
    # 初始化列表用于存储从pdf文件中提取出来的图片
    images = []
    # for循环遍历pdf文件夹中的每个pdf文件并将其转换为原始图像存入列表
    for pdf_file in sorted(os.listdir(pdf_folder_path)):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, pdf_file)
            image_list_pdf = convert_from_path(pdf_path)
            image = image_list_pdf[0]
            images.append(image)     
    return images

""" 
    裁剪图片函数
    images: 图像列表
    return: img_cropped_list，裁剪后的图像列表
"""
def crop_images(images):
    # 初始化一个列表，用于存储裁剪后的图像
    img_cropped_list = []
    left = 250 # 左边界
    top = 360    # 上边界
    right = 1340  # 右边界
    bottom = 1950 # 下边界
    for i, image in enumerate(images):
        # 将图片保存以便进行OCR识别（也可以直接对PIL.Image对象进行处理）
        image.save(temp_img_path+f"page_{i+1}.png")

        # 打开原图片开始进行裁剪
        img = Image.open(temp_img_path+f"page_{i+1}.png")
        # 确保裁剪区域的坐标不会导致图像超出范围
        left = max(0, left)
        top = max(0, top)
        right = min(img.width, right)
        bottom = min(img.height, bottom)
        # 开始裁剪
        img_cropped = img.crop((left, top, right, bottom))
        img_cropped.save(temp_img_path+f"page_{i+1}_cropped.png")
        img_cropped_list.append(img_cropped)
    return img_cropped_list
    
""" 
    使用Tesseract进行OCR识别 
    img_cropped_list: 裁剪后的图像列表
    return: text_list，识别的文本列表
""" 
def ocr(img_cropped_list):
    # 初始化一个文本列表用于存储每张图像提取出来的文本
    text_list = []
    # 用于存储两部分信息的列表
    person_info_list = []
    test_data_list = []

    for i, img_cropped in enumerate(img_cropped_list):
        text = pytesseract.image_to_string(img_cropped, lang='eng+chi_sim')
        text_list.append(text)
    
    # 分割用户信息与测试数据
    for i, text in enumerate(text_list):
        lines = text.splitlines()
        person_info = "\n".join(lines[:6])
        test_data = "\n".join(lines[6:])
        person_info_list.append(person_info)
        test_data_list.append(test_data)

    return person_info_list, test_data_list

def main():
    images = []
    images = pdf2img(pdf_folder_path)
    img_cropped_list = crop_images(images)
    person_info_list, test_data_list = ocr(img_cropped_list)
    
    for person_info in person_info_list:
        print(person_info)
    print("--------------------------------------------------------")
    for test_data in test_data_list:
        print(test_data)


if __name__ == '__main__':
    main()

REA 杨 会 琴 Wik.
住院 号 : 1973315 ase
出 生日 期 : 1957/11/30 操作 者 :
PERN : 女 Gi.
身高 160 cm 体
年 龄 Ag ee =
--------------------------------------------------------
预计 实测 %( 实 / 预 )
测试 日 其 24/3/04
测试 时 间 16:05:20
VT [L] 0.52 0.72 138.8
BF {1/min] 20.00 53. 44 267. 2
MV {L/min} 10.43 38.67 370.8
ERV [L] 0.70 0. 49 70.4
VC MAX (L] 2.59 2. 32 89.5
FVC {L] 2.48 2.17 87.4
FEV 1 [Bui] 2,0 1.64 79.3
FEV 1 % FVC [%] 75. 68
FEV 1 % VC MAX [%] 76.56 70.78 92.5
PEF 并 二 5. 06 88.6
MEF 75 {L/s] 5.10 3.87 75.9
MEF 50 [L/s] 3.43 1.43 41.7
MEF 25 (Ste) pe 0. 46 40.6
MMEF 75/25 [L/s 2.68 0.98 36.7
MVV [L/min] 84.80 56.96 67.2
FEV 1#30 [L/min] 84.80 49.26 58. 1
RV-SB [L 1.95 1.49 76.3
TLC-SB (il ae 3.49 1382)
RV%TLC-SB [%] 41.40 42. 64 103.0
FRC-SB cee 2.65) 1.98 74.8
FRC%TLC-SB [%] 55. 66 56. 70 101.9
{mmol/min/kPa] 7.11 6. 27 88.1

DLCO SB

16889648-19

713 kg

肺 功能 意见 ;

